python dct_steganography

In [14]:
import cv2
import numpy as np

def embed_text_in_dct():
    # Load the image in grayscale
    img = cv2.imread("/bmw.jpg", 0)

    # Check if the image was loaded successfully
    if img is None:
        print("❌ Error: Could not load the image at 'input/original_image.jpg'.")
        print("Please ensure the file exists and is a valid image format.")
        return # Exit the function if the image wasn't loaded

    h, w = img.shape
    message = open("/message.txt", "r").read() + chr(0)
    binary = ''.join(format(ord(c), '08b') for c in message)

    idx = 0
    for i in range(0, h - 8, 8):
        for j in range(0, w - 8, 8):
            if idx >= len(binary):
                break
            block = np.float32(img[i:i+8, j:j+8])
            dct_block = cv2.dct(block)
            # Ensure the value at dct_block[4, 4] is an integer before the modulo operation
            if int(dct_block[4, 4]) % 2 != int(binary[idx]):
                 # Ensure the value at dct_block[4, 4] is a float before adding 1
                dct_block[4, 4] = float(int(dct_block[4, 4]) + 1)
            # Ensure the output of cv2.idct is converted back to the original image data type (uint8 for typical images)
            img[i:i+8, j:j+8] = cv2.idct(dct_block).astype(np.uint8)
            idx += 1

    cv2.imwrite("/bmw.jpg", img)
    print("✅ Text successfully hidden in output/bmw.jpg")

embed_text_in_dct()

✅ Text successfully hidden in output/bmw.jpg


python lsb_steganography

In [15]:
from PIL import Image

def text_to_bin(message):
    return ''.join(format(ord(char), '08b') for char in message)

def bin_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join([chr(int(b, 2)) for b in chars])

def hide_text():
    with open("/message.txt", "r") as f:
        secret_text = f.read()

    image = Image.open("/1276475.jpg")
    binary = text_to_bin(secret_text) + '1111111111111110'  # End marker
    data = iter(image.getdata())
    new_pixels = []

    for i in range(0, len(binary), 3):
        pixel = list(next(data))
        for j in range(3):
            if i + j < len(binary):
                pixel[j] = pixel[j] & ~1 | int(binary[i + j])
        new_pixels.append(tuple(pixel))

    new_pixels.extend(data)
    image.putdata(new_pixels)
    image.save("/1276475.jpg")
    print("✅ Text successfully hidden in output/1276475.jpg")

def extract_text():
    image = Image.open("/1276475.jpg")
    binary = ''
    for pixel in image.getdata():
        for value in pixel[:3]:
            binary += str(value & 1)
            if binary[-16:] == '1111111111111110':
                message = bin_to_text(binary[:-16])
                print("🔍 Extracted message:", message)
                return

hide_text()
extract_text()


✅ Text successfully hidden in output/1276475.jpg
🔍 Extracted message: $I$IÛm¶Ûm¶Ûm¶Ûm¶Ûm¶Ûm¶Ûm¶$IÛm¶$I ~ ÿ


python hybrid_secure_steganography

In [16]:
# Install the pycryptodome library if not already installed
!pip install pycryptodome

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64
from PIL import Image

key = b"thisisasecretkey"  # 16-byte key

def encrypt_msg(text):
    cipher = AES.new(key, AES.MODE_CBC, iv=key)
    ct = cipher.encrypt(pad(text.encode(), AES.block_size))
    return base64.b64encode(ct).decode()

def decrypt_msg(enc_text):
    cipher = AES.new(key, AES.MODE_CBC, iv=key)
    ct = base64.b64decode(enc_text)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def to_bin(data):
    # This function is fine for converting the base64 string to binary
    return ''.join(format(ord(i), '08b') for i in data)

def hide_encrypted_text():
    with open("/message.txt", "r") as f:
        message = f.read()
    encrypted = encrypt_msg(message)
    # The binary representation is of the base64 encoded string
    binary = to_bin(encrypted) + '1111111111111110'
    img = Image.open("/bmw.jpg")

    # Check if the image is grayscale and convert to RGB if necessary
    if img.mode == 'L':
        img = img.convert('RGB')

    data = iter(img.getdata())
    new_pixels = []

    for i in range(0, len(binary), 3):
        pixel = list(next(data))
        for j in range(3):
            if i + j < len(binary):
                pixel[j] = pixel[j] & ~1 | int(binary[i + j])
        new_pixels.append(tuple(pixel))

    new_pixels.extend(data)
    img.putdata(new_pixels)
    # Changed output filename for clarity
    img.save("/secure_encoded.png")
    print("✅ Encrypted text hidden in /secure_encoded.png")


def extract_encrypted_text():
    # Load the image where the encrypted text was hidden
    img = Image.open("/secure_encoded.png")
    binary = ''
    # Extract the LSBs to get the binary string representation of the base64 data
    for pixel in img.getdata():
        for value in pixel[:3]:
            binary += str(value & 1)
            # Check for the end marker
            if binary[-16:] == '1111111111111110':
                # Remove the end marker
                binary_data = binary[:-16]
                # Convert the binary data back into a string of characters.
                # This string should represent the original base64 encoded data.
                extracted_chars = ''.join([chr(int(binary_data[i:i+8], 2)) for i in range(0, len(binary_data), 8)])
                # Now, decode the extracted base64 string.
                # base64.b64decode expects bytes, so encode the string to bytes using ascii.
                # This should now work because the extracted string *should* be valid base64.
                print("🔓 Decrypted Message:", decrypt_msg(extracted_chars.encode('ascii')))
                return
    # If the end marker is not found, print a message
    print("❌ Could not find the end marker in the image.")


hide_encrypted_text()
extract_encrypted_text()

✅ Encrypted text hidden in /secure_encoded.png
🔓 Decrypted Message: HI i am here the password is 2309 .
its urgent to decode the fingerprints from the deputy manager.


 PVD steganography

In [18]:
from PIL import Image
import os # Import the os module

def get_difference(pixel1, pixel2):
    return abs(pixel1 - pixel2)

def hide_text_pvd(img_path, message, output_path):
    img = Image.open(img_path).convert("L")  # grayscale
    data = list(img.getdata())
    binary = ''.join(format(ord(c), '08b') for c in message + chr(0))
    idx = 0

    for i in range(0, len(data) - 1, 2):
        if idx >= len(binary):
            break
        d = get_difference(data[i], data[i+1])
        if d < 10:
            bits_to_embed = 1
        elif d < 20:
            bits_to_embed = 2
        elif d < 40:
            bits_to_embed = 3
        else:
            bits_to_embed = 4

        if idx + bits_to_embed > len(binary):
            break

        bits = binary[idx:idx+bits_to_embed]
        val = int(bits, 2)
        new_d = val

        if data[i] > data[i+1]:
            data[i+1] = data[i] - new_d
        else:
            data[i+1] = data[i] + new_d

        idx += bits_to_embed

    # Ensure the output directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    img.putdata(data)
    img.save(output_path)
    print("✅ PVD steganography complete.")

hide_text_pvd("/1276475.jpg", open("/message.txt").read(), "output/pvd_encoded.png")

✅ PVD steganography complete.


Edge-Based LSB

In [20]:
import cv2
from PIL import Image

def to_bin(data):
    return ''.join(format(ord(i), '08b') for i in data)

def edge_lsb_hide():
    img = cv2.imread("/bmw.jpg")

    # Check if the image was loaded successfully
    if img is None:
        print("❌ Error: Could not load the image at '/bmw.jpg'.")
        print("Please ensure the file exists and is a valid image format.")
        return

    # Convert to grayscale for edge detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)

    message = open("/message.txt").read() + chr(0)
    binary = to_bin(message)

    # Re-open the image with PIL to get pixel data in a suitable format
    # Convert to RGB to ensure consistent pixel format (tuples)
    img_pil = Image.open("/bmw.jpg").convert("RGB")
    pixels = list(img_pil.getdata())
    idx = 0

    # Iterate through the edge map
    for y in range(edges.shape[0]):
        for x in range(edges.shape[1]):
            # Check if there is an edge at this location and if there's still binary data to embed
            if edges[y, x] != 0 and idx < len(binary):
                # Calculate the index in the flattened pixel list
                i = y * edges.shape[1] + x
                # Get the pixel at the current index as a list of values (e.g., [R, G, B])
                p = list(pixels[i])
                # Embed the binary data into the LSBs of the color channels (R, G, B)
                for k in range(3):
                    if idx < len(binary):
                        p[k] = p[k] & ~1 | int(binary[idx])
                        idx += 1
                # Update the pixel in the list with the modified values
                pixels[i] = tuple(p)

    # Put the modified pixel data back into the PIL image
    img_pil.putdata(pixels)
    # Save the steganographed image
    img_pil.save("output/edge_lsb_encoded.png")
    print("✅ Edge-based LSB complete.")

# Ensure the output directory exists before saving the image
import os
output_dir = "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

edge_lsb_hide()

✅ Edge-based LSB complete.


Histogram Shifting

In [21]:
from PIL import Image
import numpy as np

def histogram_shift_hide(img_path, message, output_path):
    img = Image.open(img_path).convert("L")
    arr = np.array(img)
    flat = arr.flatten()
    binary = ''.join(format(ord(c), '08b') for c in message + chr(0))
    idx = 0
    peak = 128  # Most frequent pixel usually around mid-intensity

    for i in range(len(flat)):
        if flat[i] == peak and idx < len(binary):
            flat[i] += int(binary[idx])
            idx += 1

    new_img = Image.fromarray(flat.reshape(arr.shape))
    new_img.save(output_path)
    print("✅ Histogram Shifting complete.")

histogram_shift_hide("/bmw.jpg", open("/message.txt").read(), "output/histogram_encoded.png")


✅ Histogram Shifting complete.


 DWT Steganography

In [24]:
import pywt
import cv2
import numpy as np

def dwt_hide(img_path, message, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Check if the image was loaded successfully
    if img is None:
        print(f"❌ Error: Could not load the image at '{img_path}'.")
        print("Please ensure the file exists and is a valid image format.")
        return

    # Perform DWT
    coeffs2 = pywt.dwt2(img, 'haar')
    cA, (cH, cV, cD) = coeffs2

    binary = ''.join(format(ord(c), '08b') for c in message + chr(0))
    idx = 0

    # Iterate through the detail coefficients (horizontal)
    # Embed the binary message into the LSB of the coefficients
    for i in range(cH.shape[0]):
        for j in range(cH.shape[1]):
            if idx < len(binary):
                # Convert the float coefficient to an integer before bitwise operations
                # This modifies the integer representation of the float
                cH[i, j] = float(int(cH[i, j]) & ~1 | int(binary[idx]))
                idx += 1

    # Reconstruct the image from the modified coefficients
    coeffs2 = cA, (cH, cV, cD)
    stego_img = pywt.idwt2(coeffs2, 'haar')

    # Ensure the output directory exists
    import os
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the steganographed image
    # Convert to uint8 before saving as image
    cv2.imwrite(output_path, np.uint8(stego_img))
    print(f"✅ DWT Steganography complete. Output saved to {output_path}")

# Install pywt if not already installed
!pip install pywt

# Call the function
dwt_hide("/1276475.jpg", open("/message.txt").read(), "output/dwt_encoded.jpg")

ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt
✅ DWT Steganography complete. Output saved to output/dwt_encoded.jpg
